#GNN model
I have tried to implement the text based GNN classifier from a model proposed by Huang, Dehong Ma, Sujian Li, Xiaodong Zhang and Houfeng WANGMOE of Key Lab of Computational Linguistics, Peking University, Beijing, 100871, China. This model outperformed the previous state of the art model and is computationally less expensive. Also, It was my first time with GNN and I thought this model is can be implemented easily.

In [1]:
import pandas as pd
import torch
import torch.nn as nn 
import torch.nn.functional as F
import numpy as np
from time import time
from torch.utils.data import random_split, DataLoader

Train and Test file location

In [2]:
train_file_path = '/content/drive/MyDrive/GNN/Datasets/Training_Data.01_original.xlsx'
test_file_path = '/content/drive/MyDrive/GNN/Datasets/Testing_Data_2_ (1).xlsx'

In [3]:
train = pd.read_excel(train_file_path)
test  = pd.read_excel(test_file_path)

In [4]:
#Parameter Values
embedding_size = 100
output_size = 62
p = 2
MAX_length = 100
min_freq = 2
train_validation_split = 0.8
dropout = 0.5
epochs = 100
weight_decay = 0.00
lr = 0.0005

In [5]:
training_vocab = []
for i , sentence in enumerate(train['Business Description']):
  try:
    training_vocab.extend(sentence.split())
  except:
    #skipping some empty description columns
    train = train.drop(i)
    
#saving the new training file 
train.to_excel('/content/drive/MyDrive/GNN/Datasets/Training_Data.01 (2).xlsx') 
train = pd.read_excel('/content/drive/MyDrive/GNN/Datasets/Training_Data.01 (2).xlsx', usecols=['Business Description', 'Industry Classification Tag'])

#GloveTokenizer

Glove Tokenizer to tokenize words in the description <br>
Used embedding size of 50 so each word is created into a token of 50 length



In [6]:
glove_file_path = '/content/drive/MyDrive/GNN/Datasets/glove.6B.'+str(embedding_size) + 'd.txt'

In [7]:
stoi = {'<unk>': -1, '<pad>': -2} 
itos = {-1: '<unk>', -2: '<pad>'} 
unknown = '<unk>'
Padding = '<pad>'
embedding_matrix = []
with open(glove_file_path, encoding='utf8') as f:
  for i, line in enumerate(f):
    values = line.split()
    stoi[values[0]] = i
    itos[i] = values[0]
    embedding_matrix.append([float(v) for v in values[1:]])

embedding_matrix = np.array(embedding_matrix).astype(np.float32)

In [8]:
embedding_matrix.shape

(400000, 100)

In [9]:
def encoding(sentence, stoi = stoi, itos= itos):
  try:
    sentence = sentence.split()
  except:
    None
  encoded_sentence = []
  for word in sentence:
    encoded_sentence.append(stoi.get(word, stoi['<unk>']))
  return encoded_sentence

In [10]:
def decoding(encoded_sentence, stoi = stoi, itos = itos):
  sentence = []
  for encoded_word in encoded_sentence:
    sentence.append(itos.get(encoded_word))
  return sentence

In [11]:
#sanity check
temp = "my name is harshit, <pad>, I My"
encoded_temp = encoding(temp.lower())
decoded_temp = decoding(encoded_temp)
print(encoded_temp)
print(decoded_temp)

[192, 311, 14, -1, -1, 41, 192]
['my', 'name', 'is', '<unk>', '<unk>', 'i', 'my']


In [12]:
def embedding(encoded_sentence, embedding_matrix = embedding_matrix):
  return embedding_matrix[np.array(encoded_sentence)]

In [13]:
#sanity check
embedding(encoded_temp).shape
#100 length vectors for the seven encoded words in sentence

(7, 100)

#Dataset Creation

Creating a label dictionary to map the labels

In [14]:
label_dic = dict(zip(train['Industry Classification Tag'].unique(), pd.get_dummies(train['Industry Classification Tag'].unique()).values))

In [15]:
label_dic

{'Advertising': array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8),
 'Aerospace & Defense': array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8),
 'Apparel Retail': array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8),
 'Apparel, Accessories & Luxury Goods': array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8),
 'App

Train, validation split


In [16]:
train_dataset, val_dataset = random_split(train.to_numpy(), [int(len(train)*train_validation_split), len(train)-int(len(train)*train_validation_split)])

**Building Vocabulary of our model**

In [17]:
stoi_dataset = {'<unk>': 0, '<pad>': 1}     #string to index dictionary for datasets' vocabulary
itos_dataset = {0: '<unk>', 1: '<pad>'}     #index to string dictionary basically inverse 

In [18]:
next(iter(train_dataset))

array(["IntriCon Corporation is engaged in designing, developing, engineering, manufacturing and distributing body-worn devices. The Company operates through body-worn device segment. The Company serves the body-worn device market by designing, developing, engineering and manufacturing micro-miniature products, microelectronics, micro-mechanical assemblies, complete assemblies and software solutions, primarily for the value hearing health market, the medical bio-telemetry market and the professional audio communication market. The Company has facilities in Minnesota, California, Singapore, Indonesia, the United Kingdom and Germany, and operates through its subsidiaries. The Company's product offering includes a hearing aid discount program for health plans. This program is available around the nation to health insurers, including employer-sponsored, individual and Medicare plans. The Company also has various international value hearing aid (VHA) initiatives.",
       'Health Care Equip

In [19]:
vocab_list = []
for sentence, _ in train_dataset:
  try:
    vocab_list.extend(sentence.split())
  except:
    None
#training_vocab = train['Business Description'][0].split()
print("Length of training data vocabulary: %d"%len(vocab_list))

Length of training data vocabulary: 501956


In [20]:
vocab_count = len(stoi_dataset)

for vocab in vocab_list:
  if stoi_dataset.get(vocab.lower()) is None:
    if vocab.lower() in stoi.keys():
      stoi_dataset[vocab.lower()] = vocab_count
      itos_dataset[vocab_count] = vocab.lower()
      vocab_count = vocab_count + 1
embedding_matrix_train_dataset = embedding(encoding(list(stoi_dataset.keys())))

In [21]:
stoi_dataset.keys()

dict_keys(['<unk>', '<pad>', 'corporation', 'is', 'engaged', 'in', 'manufacturing', 'and', 'distributing', 'the', 'company', 'operates', 'through', 'device', 'serves', 'market', 'by', 'engineering', 'complete', 'assemblies', 'software', 'primarily', 'for', 'value', 'hearing', 'health', 'medical', 'professional', 'audio', 'communication', 'has', 'facilities', 'united', 'kingdom', 'its', 'product', 'offering', 'includes', 'a', 'aid', 'discount', 'program', 'this', 'available', 'around', 'nation', 'to', 'including', 'individual', 'medicare', 'also', 'various', 'international', 'inc.', 'provider', 'of', 'information', 'solutions', 'both', 'traffic', 'management', 'global', 'agribusiness', 'three', 'roadway', 'transportation', 'systems', 'performance', 'sensors', 'segment', 'provides', 'vehicle', 'detection', 'products', 'intersection', 'incident', 'data', 'collection', 'consulting', 'development', 'traveler', 'intelligent', 'analytics', 'specialized', 'measurement', 'as', 'well', 'precisio

In [22]:
embedding_matrix_train_dataset.shape

(19599, 100)

**Training Dataset preperation**

In [23]:
class Dataset_creation(): # For instantiating train, validation and test dataset
    def __init__(self, node_sets, neighbor_sets, public_edge_mask, labels):
        super(Dataset_creation).__init__()
        self.node_sets = node_sets
        self.neighbor_sets = neighbor_sets
        self.public_edge_mask = public_edge_mask
        self.labels = labels

    def __getitem__(self, i):
        return torch.LongTensor(self.node_sets[i]), \
               torch.nn.utils.rnn.pad_sequence([torch.LongTensor(neighbor) for neighbor in self.neighbor_sets[i]], batch_first=True, padding_value=1), \
               self.public_edge_mask[torch.LongTensor(self.node_sets[i]).unsqueeze(-1).repeat(1, torch.nn.utils.rnn.pad_sequence([torch.LongTensor(neighbor) for neighbor in self.neighbor_sets[i]], batch_first=True, padding_value=1).shape[-1]), torch.nn.utils.rnn.pad_sequence([torch.LongTensor(neighbor) for neighbor in self.neighbor_sets[i]], batch_first=True, padding_value=1)], \
               torch.FloatTensor(self.labels[i])

    def __len__(self):
        return len(self.labels)


In [24]:
def get_neighbour_set(node_set, p = p):
  neighbour_set = []
  for i in range(len(node_set)):
    neighbour = []
    for j in range(-p, p+1):
      if 0<=i+j<len(node_set):
        neighbour.append(node_set[i+j])
    neighbour_set.append(neighbour)

  return neighbour_set

In [25]:
node_sets = [torch.tensor([stoi_dataset.get(vocab, 0) for vocab in sentence.lower().strip().split(' ')][0:MAX_length]) for sentence, _ in train_dataset]
node_sets = torch.nn.utils.rnn.pad_sequence(node_sets, batch_first=True, padding_value=1)
neighbour_sets = [get_neighbour_set(node_set, p) for node_set in node_sets]
label_sets = [label_dic[label] for _, label in train_dataset]

In [26]:
edge_stat = torch.zeros(vocab_count, vocab_count)
for node_set, neighbour_set in zip(node_sets, neighbour_sets):
  for neighbour in neighbour_set:
    for to_node in neighbour:
      edge_stat[node_set, to_node] += 1 

In [27]:
public_edge_mask = edge_stat<min_freq

In [28]:
train_d = Dataset_creation(node_sets, neighbour_sets, public_edge_mask, label_sets)

**Validation Data**

In [29]:
node_sets = [torch.tensor([stoi_dataset.get(vocab, 0) for vocab in sentence.lower().strip().split(' ')][0:MAX_length]) for sentence, _ in val_dataset]
node_sets = torch.nn.utils.rnn.pad_sequence(node_sets, batch_first=True, padding_value=1)
neighbour_sets = [get_neighbour_set(node_set, p) for node_set in node_sets]
label_sets = [label_dic[label] for _, label in val_dataset]

In [30]:
val_d = Dataset_creation(node_sets, neighbour_sets, public_edge_mask, label_sets)

#Modelling

In [31]:
temp = torch.randn((32,200,5,100))
temp.view(-1,1).shape

torch.Size([3200000, 1])

In [32]:
class MessagePassing(nn.Module):
    def __init__(self, vertice_count, input_size, out_size, dropout_rate=dropout, padding_idx=1):
        super(MessagePassing, self).__init__()
        self.vertice_count = vertice_count # |V|
        self.input_size = input_size # d
        self.out_size = out_size # c
        self.dropout_rate = dropout_rate
        self.padding_idx = padding_idx
        self.information_rate = nn.Parameter(torch.rand(self.vertice_count, 1)) # (|V|, 1), which means it is a column vector
        self.linear = nn.Linear(self.input_size, 256) # (d, c)
        self.linear2 = nn.Linear(256, self.out_size)
        self.dropout = nn.Dropout(self.dropout_rate)

    def forward(self, node_sets, embedded_node, edge_weight, embedded_neighbor_node):
        # node_sets: (batch_size, l)
        # embedded_node: (batch_size, l, d)
        # edge_weight: (batch_size, max_sentence_length, max_neighbor_count)
        # embedded_neighbor_node: (batch_size, max_sentence_length, max_neighbor_count, d)

        tmp_tensor = (edge_weight.view(-1, 1) * embedded_neighbor_node.view(-1, self.input_size)).view(embedded_neighbor_node.shape) # (batch_size, max_sentence_length, max_neighbor_count, d)
        tmp_tensor = tmp_tensor.masked_fill(tmp_tensor == 0, -1e18) # (batch_size, max_sentence_length, max_neighbor_count, d), mask for M such that masked places are marked as -1e18
        tmp_tensor = self.dropout(tmp_tensor)
        M = tmp_tensor.max(dim=2)[0] # (batch_size, max_sentence_length, d), which is same shape as embedded_node (batch_size, l, d)
        information_rate = self.information_rate[node_sets] # (batch_size, l, 1)
        information_rate = information_rate.masked_fill((node_sets == self.padding_idx).unsqueeze(-1), 1) # (batch_size, l, 1), Fill the information rate of the padding index as 1, such that new e_n = (1-i_r) * M + i_r * e_n = (1-1) * 0 + 1 * e_n = e_n (no update)
        embedded_node = (1 - information_rate) * M + information_rate * embedded_node # (batch_size, l, d)
        sum_embedded_node = embedded_node.sum(dim=1) # (batch_size, d)
        x = F.relu(self.linear(sum_embedded_node)) # (batch_size, c)
        x = F.relu(self.linear2(x))
        y = F.softmax(x, dim=1) # (batch_size, c) along the c dimension
        return y


class TextLevelGNN(nn.Module):
    def __init__(self, pretrained_embeddings, out_size=62, dropout_rate=dropout, padding_idx=1):
        super(TextLevelGNN, self).__init__()
        self.out_size = out_size # c
        self.padding_idx = padding_idx
        self.weight_matrix = nn.Parameter(torch.randn(pretrained_embeddings.shape[0], pretrained_embeddings.shape[0])) # (|V|, |V|)        
        self.embedding = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=False, padding_idx=self.padding_idx) # (|V|, d)
        self.message_passing = MessagePassing(vertice_count=pretrained_embeddings.shape[0], input_size=pretrained_embeddings.shape[1], out_size=self.out_size, dropout_rate=dropout_rate, padding_idx=self.padding_idx) # input_size: (d,); out_size: (c,)
        self.public_edge_weight = nn.Parameter(torch.randn(1, 1)) # (1, 1)

    def forward(self, node_sets, neighbor_sets, public_edge_mask):
        # node_sets: (batch_size, l)
        # neighbor_sets: (batch_size, max_sentence_length, max_neighbor_count)
        # neighbor_sets_mask: (batch_size, max_sentence_length, max_neighbor_count) (no need)
        # public_edge_mask: (batch_size, max_sentence_length, max_neighbor_count)

        embedded_node = self.embedding(node_sets) # (batch_size, l, d)
        edge_weight = model.weight_matrix[node_sets.unsqueeze(2).repeat(1, 1, neighbor_sets.shape[-1]), neighbor_sets] # (batch_size, max_sentence_length, max_neighbor_count), neighbor_sets.shape[-1]: eg p=2, this expression=5; p=3, this expression=7. This is to first make node_sets to have same shape with neighbor_sets, then just do 1 query instead of 32*100 queries to speed up performance
        a = edge_weight * ~public_edge_mask # (batch_size, max_sentence_length, max_neighbor_count)
        b = self.public_edge_weight.unsqueeze(2).expand(1, public_edge_mask.shape[-2], public_edge_mask.shape[-1]) * public_edge_mask # (batch_size, max_sentence_length, max_neighbor_count)
        edge_weight = a + b # (batch_size, max_sentence_length, max_neighbor_count)
        embedded_neighbor_node = self.embedding(neighbor_sets) # (batch_size, max_sentece_length, max_neighbor_count, d)

        # Apply mask to edge_weight, to mask and cut-off any relationships to the padding nodes
        edge_weight = edge_weight.masked_fill((node_sets.unsqueeze(2).repeat(1, 1, neighbor_sets.shape[-1]) == self.padding_idx) | (neighbor_sets == self.padding_idx), 0) # (batch_size, max_sentence_length, max_neighbor_count)
        x = self.message_passing(node_sets, embedded_node, edge_weight, embedded_neighbor_node) # (batch_size, c)
        return x

In [33]:
train_loader = DataLoader(train_d, batch_size=32,shuffle=True)
val_loader = DataLoader(val_d, batch_size=32, shuffle=True)

In [34]:
device = 'cuda'

In [35]:
model = TextLevelGNN(pretrained_embeddings=torch.tensor(embedding_matrix_train_dataset), dropout_rate=dropout).to(device)

The first loop with 500 epochs and 0.001 learning rate

In [ ]:
criterion = nn.BCELoss()

training = {}
validation = {}
training['accuracy'] = []
training['loss'] = []
validation['accuracy'] = []
validation['loss'] = []

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
for epoch in range(epochs):
    model.train()
    train_loss = 0
    train_correct_items = 0
    previous_epoch_timestamp = time()
    

    for i, (node_sets, neighbor_sets, public_edge_masks, labels) in enumerate(train_loader):
        #print('Finished batch:', i)
        node_sets = node_sets.to(device)
        neighbor_sets = neighbor_sets.to(device)
        public_edge_masks = public_edge_masks.to(device)
        labels = labels.to(device)
        prediction = model(node_sets, neighbor_sets, public_edge_masks)
        loss = criterion(prediction, labels).to(device)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_correct_items += (prediction.argmax(dim=1) == labels.argmax(dim=1)).sum().item()
    train_accuracy = train_correct_items / len(train_dataset)

    
    model.eval()
    validation_loss = 0
    validation_correct_items = 0
    for i, (node_sets, neighbor_sets, public_edge_masks, labels) in enumerate(val_loader):
      node_sets = node_sets.to(device)
      neighbor_sets = neighbor_sets.to(device)
      public_edge_masks = public_edge_masks.to(device)
      labels = labels.to(device)
      prediction = model(node_sets, neighbor_sets, public_edge_masks)
      loss = criterion(prediction, labels).to(device)
      validation_loss += loss.item()
      validation_correct_items += (prediction.argmax(dim=1) == labels.argmax(dim=1)).sum().item()
    validation_accuracy = validation_correct_items / len(val_dataset)
    
    print(f'Epoch: {epoch+1}, Training Loss: {train_loss:.4f}, Validation Loss: {validation_loss:.4f}, Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {validation_accuracy:.4f}, Time Used: {time()-previous_epoch_timestamp:.2f}s')
    training['accuracy'].append(train_accuracy)
    training['loss'].append(train_loss)
    validation['accuracy'].append(validation_accuracy)
    validation['loss'].append(validation_loss)

    

Epoch: 1, Training Loss: 13.2365, Validation Loss: 4.4959, Training Accuracy: 0.0681, Validation Accuracy: 0.1323, Time Used: 44.17s
Epoch: 2, Training Loss: 10.2061, Validation Loss: 4.2995, Training Accuracy: 0.1402, Validation Accuracy: 0.1633, Time Used: 44.02s
Epoch: 3, Training Loss: 9.8951, Validation Loss: 4.2729, Training Accuracy: 0.1559, Validation Accuracy: 0.1644, Time Used: 43.76s
Epoch: 4, Training Loss: 9.7033, Validation Loss: 4.1423, Training Accuracy: 0.1720, Validation Accuracy: 0.1821, Time Used: 43.87s
Epoch: 5, Training Loss: 9.5621, Validation Loss: 4.1319, Training Accuracy: 0.1877, Validation Accuracy: 0.1937, Time Used: 44.40s
Epoch: 6, Training Loss: 9.4111, Validation Loss: 4.0688, Training Accuracy: 0.2028, Validation Accuracy: 0.1976, Time Used: 43.82s
Epoch: 7, Training Loss: 9.3197, Validation Loss: 3.9900, Training Accuracy: 0.2176, Validation Accuracy: 0.2308, Time Used: 43.66s
Epoch: 8, Training Loss: 9.1108, Validation Loss: 3.9814, Training Accurac

Second iteration with learning rate reduced to 0.0005 and 200 epochs

In [ ]:
model.to('cuda')
optimizer = torch.optim.Adam(model.parameters(), lr=lr/2, weight_decay=weight_decay)
for epoch in range(200):
    model.train()
    train_loss = 0
    train_correct_items = 0
    previous_epoch_timestamp = time()
    

    for i, (node_sets, neighbor_sets, public_edge_masks, labels) in enumerate(train_loader):
        #print('Finished batch:', i)
        node_sets = node_sets.to(device)
        neighbor_sets = neighbor_sets.to(device)
        public_edge_masks = public_edge_masks.to(device)
        labels = labels.to(device)
        prediction = model(node_sets, neighbor_sets, public_edge_masks)
        loss = criterion(prediction, labels).to(device)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_correct_items += (prediction.argmax(dim=1) == labels.argmax(dim=1)).sum().item()
    train_accuracy = train_correct_items / len(train_dataset)

    
    model.eval()
    validation_loss = 0
    validation_correct_items = 0
    for i, (node_sets, neighbor_sets, public_edge_masks, labels) in enumerate(val_loader):
      node_sets = node_sets.to(device)
      neighbor_sets = neighbor_sets.to(device)
      public_edge_masks = public_edge_masks.to(device)
      labels = labels.to(device)
      prediction = model(node_sets, neighbor_sets, public_edge_masks)
      loss = criterion(prediction, labels).to(device)
      validation_loss += loss.item()
      validation_correct_items += (prediction.argmax(dim=1) == labels.argmax(dim=1)).sum().item()
    validation_accuracy = validation_correct_items / len(val_dataset)
    
    print(f'Epoch: {epoch+1}, Training Loss: {train_loss:.4f}, Validation Loss: {validation_loss:.4f}, Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {validation_accuracy:.4f}, Time Used: {time()-previous_epoch_timestamp:.2f}s')
    training['accuracy'].append(train_accuracy)
    training['loss'].append(train_loss)
    validation['accuracy'].append(validation_accuracy)
    validation['loss'].append(validation_loss)


In [ ]:
model.to('cuda')
optimizer = torch.optim.Adam(model.parameters(), lr=lr/4, weight_decay=weight_decay)
for epoch in range(100):
    model.train()
    train_loss = 0
    train_correct_items = 0
    previous_epoch_timestamp = time()
    

    for i, (node_sets, neighbor_sets, public_edge_masks, labels) in enumerate(train_loader):
        #print('Finished batch:', i)
        node_sets = node_sets.to(device)
        neighbor_sets = neighbor_sets.to(device)
        public_edge_masks = public_edge_masks.to(device)
        labels = labels.to(device)
        prediction = model(node_sets, neighbor_sets, public_edge_masks)
        loss = criterion(prediction, labels).to(device)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_correct_items += (prediction.argmax(dim=1) == labels.argmax(dim=1)).sum().item()
    train_accuracy = train_correct_items / len(train_dataset)

    
    model.eval()
    validation_loss = 0
    validation_correct_items = 0
    for i, (node_sets, neighbor_sets, public_edge_masks, labels) in enumerate(val_loader):
      node_sets = node_sets.to(device)
      neighbor_sets = neighbor_sets.to(device)
      public_edge_masks = public_edge_masks.to(device)
      labels = labels.to(device)
      prediction = model(node_sets, neighbor_sets, public_edge_masks)
      loss = criterion(prediction, labels).to(device)
      validation_loss += loss.item()
      validation_correct_items += (prediction.argmax(dim=1) == labels.argmax(dim=1)).sum().item()
    validation_accuracy = validation_correct_items / len(val_dataset)
    
    print(f'Epoch: {epoch+1}, Training Loss: {train_loss:.4f}, Validation Loss: {validation_loss:.4f}, Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {validation_accuracy:.4f}, Time Used: {time()-previous_epoch_timestamp:.2f}s')
    training['accuracy'].append(train_accuracy)
    training['loss'].append(train_loss)
    validation['accuracy'].append(validation_accuracy)
    validation['loss'].append(validation_loss)


#Testing


creating testing dataset


In [ ]:
node_sets = [torch.tensor([stoi_dataset.get(vocab, 0) for vocab in sentence.lower().strip().split(' ')][0:MAX_length]) for _, sentence in test.to_numpy()]
node_sets = torch.nn.utils.rnn.pad_sequence(node_sets, batch_first=True, padding_value=1)
neighbour_sets = [get_neighbour_set(node_set, p) for node_set in node_sets]

In [ ]:
model.to('cpu')

In [ ]:
test_d = Dataset_creation(node_sets, neighbour_sets, public_edge_mask, label_sets[0:772]) #assinging random labels to 
test_loader = DataLoader(test_d, batch_size=1024, shuffle=False)
for i, (node_sets, neighbor_sets, public_edge_masks, labels) in enumerate(test_loader):
  label_test = model(node_sets, neighbor_sets, public_edge_masks)

In [ ]:
label_test = label_test.argmax(dim = 1)

In [ ]:
inv_label_dic = {}
for temp in label_dic.keys():
  inv_label_dic[label_dic[temp].argmax()] = temp

In [ ]:
temp = []
for item in label_test:
  temp.append(inv_label_dic[int(item)])

In [ ]:
temp_df = pd.DataFrame({'Industry Classification Tag':temp})

In [ ]:
_ = pd.concat([test, temp_df], axis = 1)

In [ ]:
_.to_excel('/content/drive/MyDrive/GNN/Datasets/Prediction.xlsx')